### Assignment #5: Basic UI--Sarrah Abdulali

DS4003 | Spring 2024

Objective: Practice buidling basic UI components in Dash. 

Task: Build an app that contains the following components user the gapminder dataset: `gdp_pcap.csv`. [Info](https://www.gapminder.org/gdp-per-capita/)

UI Components:
A dropdown menu that allows the user to select `country`
- The dropdown should allow the user to select multiple countries
- The options should populate from the dataset (not be hard-coded)
A slider that allows the user to select `year`
- The slider should allow the user to select a range of years
- The range should be from the minimum year in the dataset to the maximum year in the dataset
A graph that displays the `gdpPercap` for the selected countries over the selected years
- The graph should display the gdpPercap for each country as a line
- Each country should have a unique color
- The graph should have a title and axis labels in reader friendly format
 

(2) Write Callback functions for the slider and dropdown to interact with the graph

This means that when a user updates a widget the graph should update accordingly.
The widgets should be independent of each other. 
Layout:
- Use a stylesheet
- There should be a title at the top of the page
- There should be a description of the data and app below the title (3-5 sentences)
- The dropdown and slider should be side by side above the graph and take up the full width of the page
- The graph should be below the dropdown and slider and take up the full width of the page
Submission:
- Deploy your app on Render. 
- In Canvas, submit the URL to your public Github Repo (made specifically for this assignment)
- The readme in your GitHub repo should contain the URL to your Render page. 


**For help you may use the web resources and pandas documentation. No co-pilot or ChatGPT.**

In [10]:
#import dependencies
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

In [11]:
#read in the dataset and then see first five rows of the dataframe df
df = pd.read_csv('C:/Users/Student/Downloads/gdp_pcap.csv')
df.head()

,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,599,599,599,599,599,599,599,599,599,...,4800,4910,5030,5150,5270,5390,5520,5650,5780,5920
1,Angola,465,466,469,471,472,475,477,479,481,...,24.8k,25.3k,25.9k,26.4k,26.9k,27.4k,28k,28.5k,29.1k,29.6k
2,Albania,585,587,588,590,592,593,595,597,598,...,54k,54.6k,55.2k,55.8k,56.4k,56.9k,57.5k,58.1k,58.7k,59.2k
3,Andorra,1710,1710,1710,1720,1720,1720,1730,1730,1730,...,79.3k,79.5k,79.8k,80.1k,80.4k,80.7k,81k,81.2k,81.5k,81.8k
4,UAE,1420,1430,1430,1440,1450,1450,1460,1460,1470,...,92.5k,92.6k,92.6k,92.7k,92.8k,92.9k,92.9k,93k,93.1k,93.1k


In [12]:
# conversion function to thousands, with check for type
def convert_k_to_thousand(value):
    if isinstance(value, str) and value.endswith('k'):
        return float(value[:-1]) * 1000
    return value

# function to convert all values to float values, otherwise return original value
def try_convert_to_float(x):
    try:
        return float(x)
    except ValueError:
        return x

# apply function to each element in df
df = df.applymap(try_convert_to_float)

# apply the above conversion only to specific columns excluding the first one since it has countries
for column in df.columns[1:]:  # exclude first column by slicing df.columns[1:]
    df[column] = df[column].apply(convert_k_to_thousand)


df.head()



,country,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,Afghanistan,599.0,599.0,599.0,599.0,599.0,599.0,599.0,599.0,599.0,...,4800.0,4910.0,5030.0,5150.0,5270.0,5390.0,5520.0,5650.0,5780.0,5920.0
1,Angola,465.0,466.0,469.0,471.0,472.0,475.0,477.0,479.0,481.0,...,24800.0,25300.0,25900.0,26400.0,26900.0,27400.0,28000.0,28500.0,29100.0,29600.0
2,Albania,585.0,587.0,588.0,590.0,592.0,593.0,595.0,597.0,598.0,...,54000.0,54600.0,55200.0,55800.0,56400.0,56900.0,57500.0,58100.0,58700.0,59200.0
3,Andorra,1710.0,1710.0,1710.0,1720.0,1720.0,1720.0,1730.0,1730.0,1730.0,...,79300.0,79500.0,79800.0,80100.0,80400.0,80700.0,81000.0,81200.0,81500.0,81800.0
4,UAE,1420.0,1430.0,1430.0,1440.0,1450.0,1450.0,1460.0,1460.0,1470.0,...,92500.0,92600.0,92600.0,92700.0,92800.0,92900.0,92900.0,93000.0,93100.0,93100.0


In [13]:
# explore the dataset using pandas
# get the number of rows and columns
df.shape

(195, 302)

In [14]:
# data types of the columns
df.dtypes

country     object
1800       float64
1801       float64
1802       float64
1803       float64
            ...   
2096       float64
2097       float64
2098       float64
2099       float64
2100       float64
Length: 302, dtype: object

In [15]:
# distinct values in the name column
df['country'].unique()

array(['Afghanistan', 'Angola', 'Albania', 'Andorra', 'UAE', 'Argentina',
       'Armenia', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bolivia', 'Brazil',
       'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.',
       'Congo, Rep.', 'Colombia', 'Comoros', 'Cape Verde', 'Costa Rica',
       'Cuba', 'Cyprus', 'Czech Republic', 'Germany', 'Djibouti',
       'Dominica', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador',
       'Egypt', 'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'Finland',
       'Fiji', 'France', 'Micronesia, Fed. Sts.', 'Gabon', 'UK',
       'Georgia', 'Ghana', 'Guinea', 'Gambia', 'Guinea-Bissau',
       'Equatorial Guinea', 'Greece', 'Grenada', 'Guatemala', 'Guy

In [16]:
# how many observations for each distinct value in the name column
df['country'].value_counts()

country
Afghanistan          1
Romania              1
Namibia              1
Niger                1
Nigeria              1
                    ..
Equatorial Guinea    1
Greece               1
Grenada              1
Guatemala            1
Zimbabwe             1
Name: count, Length: 195, dtype: int64

ONTO THE DASH

(1) UI Components:
A dropdown menu that allows the user to select `country`
- The dropdown should allow the user to select multiple countries
- The options should populate from the dataset (not be hard-coded)
A slider that allows the user to select `year`
- The slider should allow the user to select a range of years
- The range should be from the minimum year in the dataset to the maximum year in the dataset
A graph that displays the `gdpPercap` for the selected countries over the selected years
- The graph should display the gdpPercap for each country as a line
- Each country should have a unique color
- The graph should have a title and axis labels in reader friendly format
 

(2) Write Callback functions for the slider and dropdown to interact with the graph

This means that when a user updates a widget the graph should update accordingly.
The widgets should be independent of each other. 
Layout:
- Use a stylesheet
- There should be a title at the top of the page
- There should be a description of the data and app below the title (3-5 sentences)
- The dropdown and slider should be side by side above the graph and take up the full width of the page
- The graph should be below the dropdown and slider and take up the full width of the page

In [17]:
# import external stylesheet using class code below

stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] # load the CSS stylesheet

app = Dash(__name__, external_stylesheets=stylesheets) # initialize the app
server = app.server

In [18]:
# extract the year columns only (since 'country' is technically the first column in df)
year_cols = df.columns.tolist()[1:]  # convert it to a list and skip the first 'country' column

# convert year columns to integers and get the minimum and maximum year
min_yr = int(year_cols[0])
max_yr = int(year_cols[-1])

# callback to update the graph based on user inputs from dropdown and slider
@app.callback(
    Output('gdp-per-capita-graph', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('year-range-slider', 'value')]
)
def update_graph(selected_countries, year_range):
    # filter the DataFrame based on selected countries and year range
    selections = df[df['country'].isin(selected_countries)]
    # melt the dataframe to have a long format for plotting with plotly
    selections = selections.melt(id_vars=['country'], value_vars=year_cols, var_name='Year', value_name='GDP per Capita')
    # filter the melted dataframe for the selected year range
    selections = selections[selections['Year'].astype(int).between(year_range[0], year_range[1])]
    
    # create the figure with plotly express line chart for the selected data
    fig = px.line(
        selections, 
        x='Year', 
        y='GDP per Capita', 
        color='country',
        title='GDP per Capita Over Selected Years',
        labels={'GDP per Capita': 'GDP per Capita (current US $)'}
    )
    
    return fig

app.layout = html.Div([
    # app title
    html.H1('GDP per Capita Analysis of Different Countries  Over Time', style={'textAlign': 'center'}),
    # app description in 3 sentences
    html.P('''
        This application allows for interactive analysis of GDP per Capita data for over 190 countries in the world. 
        You can see how GDP per Capita has changed over time if you choose to select some countries in the dropdown and a year range in the slider. 
        The line graph gives a clear visual of trends and comparisons across different nations!
    ''', style={'textAlign': 'center'}),
    # container for dropdown + slider
    html.Div([
        # dropdown to select countries!
        html.Div([
            dcc.Dropdown(
                options=[{'label': country, 'value': country} for country in df['country']],
                value=df['country'].tolist(),  # default value as all countries!
                multi=True,  # allow multiple countries' selections
                id='country-dropdown'
            )
        ], style={'display': 'inline-block', 'width': '49%'}),
        # slider for selecting year range!
        html.Div([
            dcc.RangeSlider(
                id='year-range-slider',
                min=min_yr,
                max=max_yr,
                value=[min_yr, max_yr],  # default value is the full range
                marks= {year: str(year) if year % 100 == 0 else '' for year in range(min_yr, max_yr, 50)},  # for user friendliness, tick marks every 50 years and a year text label every 100 years
                step=1,  # slider moves in increments of 1 year
                tooltip={"placement": "bottom", "always_visible":True},
                allowCross=False,  # prevents the sliders from crossing over each other / user can't select invalid range
            )
        ], style={'display': 'inline-block', 'width': '49%'}),
    ], style={'display': 'flex', 'justifyContent': 'space-between', 'padding': '10px'}),
    # container for graph!
    html.Div([
        dcc.Graph(id='gdp-per-capita-graph')
    ], style={'width': '100%', 'padding': '10px'})
])

# run the app
if __name__ == '__main__':
    #appy.run(jupyter_mode='tab', debug=True)
    # i changed the code above to reflect what professor wanted us to do!
    app.run_server(debug=True) #run the server